# Homework 01 - Compression

Contact: David C. Schedl (david.schedl@fh-hagenberg.at)

Note: this is the starter pack for the **Visual Computing** homework. You do not need to use this template!

## Task:
Implement your image compression algorithm by exploiting the frequency domain. 

When you look at the spectra you will notice that some frequencies have very low-coefficients and thus can be easily removed. How much can be removed before the image quality gets pretty bad?
Think about a way of encoding which frequencies you can remove and what you keep? 

Analyze your algorithm on a set of images and evaluate your results concerning quality (e.g., MSE) and size reduction achieved (how many bits could be saved?). Can you make image quality / size reduction a parameter? 

Ideally you use (a large) set of different evaluation images (e.g., showing artificial content, buildings, nature, animals, objects, colors, …). 
*Hint:* Work with low-res images to speed up computation and avoid huge storage requirements.


**Further comments/hints:**

*   You do not need to come up with a super compression algorithm! It is mostly about getting into the topic.
*   Be creative! 🤔
*   Think about the problem, solve it, and critically evaluate your solution.
*   Summarize your ideas and findings in the report. 



**Have fun!** 😸


### Example Dataset: CIFAR 10

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. It was originally used for Machine Learning, but we can use it for our compression experiments.
For details, see [this website](https://www.cs.toronto.edu/~kriz/cifar.html)!

**Note: you do not need to use CIFAR-10! Any other dataset is fine.** 

In [ ]:
# This python package helps you to work with CIFAR-10
!pip install git+https://github.com/schedldave/py-cifar10
import cifar10


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

label_map = cifar10.image_label_map() # dictionary to map label integer values to text. i.e., {0: 'airplane', 1: 'automobile', 2: ...}

g = cifar10.data_batch_generator() # CIFAR-10 image generator
image, label = next(g)
# image # numpy array of an image, which is of shape 32 x 32 x 3
# label # integer value of the image label

plt.imshow(image)
plt.title(label_map.get(label))
plt.show()

# count all images
count = 0
count_labels = np.zeros(shape=(len(label_map),), dtype=int)
for image, label in cifar10.data_batch_generator():
  count_labels[label] += 1
  count += 1

print(f'The test set contains {count} images \nin the following categories:')
for i, per_label in enumerate(count_labels):
  print(f'  {label_map.get(i)}: {per_label} images')


### Frequencies: DFT and DCT

The Discrete Fourier Transformation and the Discrete Cosine Transfrom convert a signal into its frequencies. Unlike the Fast Fourier Transformation (FFT) their results are real numbered, which makes it easier to work with them. 

The idea and the principle is similar to an FFT. 

Below you will find example usages for OpenCV's DFT and DCT implementations:

In [ ]:
import cv2

# use a CIFAR-10 image and convert it to grayscale
gray = cv2.cvtColor(cv2.cvtColor(image,cv2.COLOR_BGR2RGB), cv2.COLOR_RGB2GRAY)

# Transform the image to frequency domain using FFT/DFT
f = cv2.dft(gray.astype(np.double))
print(repr(f.dtype)) # The DFT produces a real numbered result

plt.figure(figsize=(15,10))
plt.subplot(131), plt.imshow(gray, cmap = 'gray'), plt.title('Input Image')
plt.subplot(132), plt.imshow(f), plt.title('DFT Spectrum')
plt.subplot(133), plt.imshow(cv2.idft(f, flags=cv2.DFT_SCALE), cmap = 'gray'), plt.title('Inverse DFT')

plt.show()

In [ ]:
# use a CIFAR-10 image and convert it to grayscale
gray = cv2.cvtColor(cv2.cvtColor(image,cv2.COLOR_BGR2RGB), cv2.COLOR_RGB2GRAY)

# Transform the image to frequency domain using DCT
d = cv2.dct(gray.astype(np.double)) # NOTE: OpenCV's dct needs a floating point data type!
print(repr(d.dtype)) # The DCT uses only real numbers!

plt.figure(figsize=(15,10))
plt.subplot(131), plt.imshow(gray, cmap = 'gray'), plt.title('Input Image')
plt.subplot(132), plt.imshow(d), plt.title('DCT Spectrum')
plt.subplot(133), plt.imshow(cv2.idct(d), cmap = 'gray'), plt.title('Inverse DCT')

plt.show()

In [ ]:
def MSE(A,B):
  """compute the mean squared error (MSE) between numpy array A and B
  """
  return ((A - B)**2).mean(axis=None)

print( f'MSE: {MSE(gray, cv2.idct(d))}' ) # should be very low!
print( f'MSE: {MSE(gray, cv2.idft(f, flags=cv2.DFT_SCALE))}' ) # should be very low!